# Asignment 3
PHYS8905 - Zachary Milgate (mqg10019180)

## Question 1
### a) 
Since there are 8 items to be searched this can be performed on $2^n = 8$ qubits. Therefore $n=3$ qubits are needed.   
\begin{align}
r &\approx \frac{\pi}{4}\sqrt{2^n} - \frac{1}{2}\\
    &= 1.7214\\
k &= \lceil r \rceil\\
    &= 2
\end{align}
Grover's algorithm will require 2 iterations to find the single solution.

### b)


In [1]:
from qutip import *
import numpy as np

In [9]:
groverCirc = circuit.QubitCircuit(4, reverse_states=False)
groverCirc.add_gate("H", targets=[0,1,2,3])
groverCirc.add_gate("X", targets=[0])
groverCirc.add_gate("SNOT", targets=[0,1,2,3])
print(q)
rx(pi)

ValueError: Gate SNOT requires one target